<a href="https://colab.research.google.com/github/Uniholder/Specialization-Yandex-MIPT/blob/main/6_Final_projects/2_churn_prediction/week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir data
!wget -O data/orange_small_churn_data.train https://stepik.org/media/attachments/lesson/90608/_847b6cd041836e8fa914243e83d898fb_orange_small_churn_data.train
!wget -O data/orange_small_churn_labels.train https://stepik.org/media/attachments/lesson/90608/_ce5e9e4280a8856218f1886f007de2a4_orange_small_churn_labels.train

--2022-08-11 15:19:45--  https://stepik.org/media/attachments/lesson/90608/_847b6cd041836e8fa914243e83d898fb_orange_small_churn_data.train
Resolving stepik.org (stepik.org)... 13.69.75.239
Connecting to stepik.org (stepik.org)|13.69.75.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23722578 (23M) [application/octet-stream]
Saving to: ‘data/orange_small_churn_data.train’

data/orange_small_c 100%[===================>]  22.62M  7.49MB/s    in 3.0s    

2022-08-11 15:19:49 (7.49 MB/s) - ‘data/orange_small_churn_data.train’ saved [23722578/23722578]

--2022-08-11 15:19:50--  https://stepik.org/media/attachments/lesson/90608/_ce5e9e4280a8856218f1886f007de2a4_orange_small_churn_labels.train
Resolving stepik.org (stepik.org)... 13.69.75.239
Connecting to stepik.org (stepik.org)|13.69.75.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117024 (114K) [application/octet-stream]
Saving to: ‘data/orange_small_churn_labels.train’

data/ora

# Входные данные

In [2]:
import pandas as pd
import os
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

PATH_TO_DATA = 'data'

In [10]:
df_raw = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_data.train'))
labels = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_labels.train'), header=None, squeeze=True)
df_raw['target'] = labels
df_raw.shape

(40000, 231)

In [4]:
num_vars = df_raw.columns[:190]
cat_vars = df_raw.columns[190:]

In [15]:
df = df_raw.dropna(axis=1, how='all')
df.shape

(40000, 213)

Отделим hold-out dataset

In [18]:
df_train, df_ho = train_test_split(df, test_size=5000)
df_train.shape, df_ho.shape

((35000, 213), (5000, 213))

In [ ]:
# df_ho.to_csv(os.path.join(PATH_TO_DATA, 'hold_out_dataset.csv'))

In [19]:
cols = df_train.drop('target', axis=1).columns

Preprocessing

In [22]:
%%time
si = SimpleImputer(strategy='most_frequent')
df_train_imp = pd.DataFrame(si.fit_transform(df_train[cols]), columns=cols)

CPU times: user 3.05 s, sys: 450 ms, total: 3.5 s
Wall time: 4.15 s


In [23]:
oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
df_train_enc = pd.DataFrame(oe.fit_transform(df_train_imp), columns=cols)

In [24]:
df_train_enc['Var229'].value_counts()

0.0    28066
1.0     6884
3.0       25
2.0       25
Name: Var229, dtype: int64

# Модель

In [25]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 92 kB/s 


In [26]:
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit, cross_validate
from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report

from catboost import CatBoostClassifier

In [52]:
X, y = df_train_enc, df_train['target']
cv = StratifiedShuffleSplit(random_state=0)

In [29]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

## LogisticRegression

In [53]:
cv_results = cross_validate(estimator=LogisticRegression(random_state=0), X=X_train, y=y_train, n_jobs=-1, scoring='roc_auc', cv=cv)

In [54]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.53916896, 0.57939327, 0.55839309, 0.55926457, 0.54898143,
        0.55489244, 0.59417724, 0.55967641, 0.59371943, 0.55003126]),
 0.5637698106269535)

In [33]:
lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)
print(classification_report(y_val, lr.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.93      1.00      0.96      6495
           1       0.00      0.00      0.00       505

    accuracy                           0.93      7000
   macro avg       0.46      0.50      0.48      7000
weighted avg       0.86      0.93      0.89      7000



## RidgeClassifier

In [55]:
cv_results = cross_validate(estimator=RidgeClassifier(random_state=0), X=X_train, y=y_train, n_jobs=-1, scoring='roc_auc', cv=cv)

In [56]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.64190108, 0.65524177, 0.67181283, 0.64220813, 0.69833977,
        0.66697739, 0.69085126, 0.67226696, 0.64605258, 0.63243427]),
 0.6618086045228903)

In [38]:
rc = RidgeClassifier(random_state=0)
rc.fit(X_train, y_train)
print(classification_report(y_val, rc.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.93      1.00      0.96      6495
           1       0.00      0.00      0.00       505

    accuracy                           0.93      7000
   macro avg       0.46      0.50      0.48      7000
weighted avg       0.86      0.93      0.89      7000



## SGDClassifier

In [57]:
cv_results = cross_validate(estimator=SGDClassifier(random_state=0), X=X_train, y=y_train, n_jobs=-1, scoring='roc_auc', cv=cv)

In [58]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.48678617, 0.52744254, 0.52198198, 0.48697187, 0.44250965,
        0.57477661, 0.47759147, 0.47138077, 0.5101765 , 0.49104983]),
 0.49906674020959735)

In [41]:
sgdc = SGDClassifier(random_state=0)
sgdc.fit(X_train, y_train)
print(classification_report(y_val, sgdc.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.93      1.00      0.96      6495
           1       0.00      0.00      0.00       505

    accuracy                           0.93      7000
   macro avg       0.46      0.50      0.48      7000
weighted avg       0.86      0.93      0.89      7000



### SGDClassifier log loss

In [59]:
cv_results = cross_validate(estimator=SGDClassifier(random_state=0, loss='log'), X=X_train, y=y_train, n_jobs=-1, scoring='roc_auc', cv=cv)

In [60]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.46446406, 0.50663725, 0.50952381, 0.48885457, 0.49853833,
        0.5418827 , 0.50044126, 0.5294613 , 0.51034749, 0.50777533]),
 0.505792608935466)

In [61]:
sgdc = SGDClassifier(random_state=0, loss='log')
sgdc.fit(X_train, y_train)
print(classification_report(y_val, sgdc.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.93      0.73      0.82      6495
           1       0.08      0.30      0.13       505

    accuracy                           0.70      7000
   macro avg       0.51      0.52      0.47      7000
weighted avg       0.87      0.70      0.77      7000



## RandomForestClassifier

In [62]:
rfc = RandomForestClassifier(oob_score=True)

In [63]:
rfc.fit(X_train, y_train)

RandomForestClassifier(oob_score=True)

In [64]:
rfc.oob_score_

0.9249285714285714

In [65]:
%%time
cv_results = cross_validate(estimator=RandomForestClassifier(random_state=0), X=X_train, y=y_train, n_jobs=-1, scoring='roc_auc', cv=cv)

CPU times: user 688 ms, sys: 111 ms, total: 799 ms
Wall time: 1min 24s


In [67]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.6497619 , 0.66683949, 0.65268524, 0.60917816, 0.68123828,
        0.62669792, 0.6875023 , 0.67559294, 0.65121254, 0.61691671]),
 0.6517625482625482)

In [66]:
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)
print(classification_report(y_val, rfc.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.93      1.00      0.96      6495
           1       0.00      0.00      0.00       505

    accuracy                           0.93      7000
   macro avg       0.46      0.50      0.48      7000
weighted avg       0.86      0.93      0.89      7000



## GradientBoostingClassifier

In [ ]:
%%time
cross_val_score(estimator=GradientBoostingClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='roc_auc')

CPU times: user 1.8 s, sys: 213 ms, total: 2.01 s
Wall time: 5min 4s


array([0.72750033, 0.73476867, 0.73284527, 0.70830834, 0.7215527 ,
       0.73489524, 0.72966326])

In [ ]:
%%time
cross_val_score(estimator=GradientBoostingClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='f1')

CPU times: user 965 ms, sys: 116 ms, total: 1.08 s
Wall time: 5min 6s


array([0.01515152, 0.00763359, 0.        , 0.00754717, 0.01509434,
       0.        , 0.00749064])

In [ ]:
%%time
cross_val_score(estimator=GradientBoostingClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='precision')

CPU times: user 970 ms, sys: 128 ms, total: 1.1 s
Wall time: 5min 8s


array([0.        , 0.33333333, 0.2       , 0.6       , 0.33333333,
       0.66666667, 0.25      ])

In [ ]:
%%time
cross_val_score(estimator=GradientBoostingClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='recall')

CPU times: user 950 ms, sys: 125 ms, total: 1.08 s
Wall time: 5min 6s


array([0.00383142, 0.00383142, 0.00766284, 0.        , 0.01149425,
       0.        , 0.        ])

In [ ]:
gbc = GradientBoostingClassifier(random_state=2179)
gbc.fit(X_tr, y_tr)
print(classification_report(y_val, gbc.predict(X_val)))

              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      8069
         1.0       0.50      0.00      0.01       681

    accuracy                           0.92      8750
   macro avg       0.71      0.50      0.48      8750
weighted avg       0.89      0.92      0.89      8750



# Тест

In [ ]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
df_test = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_test_data.csv'), index_col=0)
df_test

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
ID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,1225.0,7.0,NaN,NaN,NaN,...,zCkv,APgdzOv,jySVZNlOJy,NaN,ELof,xb3V,6fzt,Zy3gnGM,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,896.0,14.0,NaN,NaN,NaN,...,oslk,IIvC99a,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,791.0,7.0,NaN,NaN,NaN,...,oslk,6YSocsg,LM8l689qOp,NaN,kG3k,rgKb,RAYp,F2FyR07IdsN7I,mj86,NaN
3,NaN,NaN,NaN,NaN,NaN,2296.0,7.0,NaN,NaN,NaN,...,oslk,5nQ7A2G,jySVZNlOJy,NaN,kG3k,rgKb,RAYp,F2FyR07IdsN7I,am7c,NaN
4,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,...,oslk,MI8s5nE,LM8l689qOp,NaN,NaN,7P5s,RAYp,F2FyR07IdsN7I,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,NaN,NaN,NaN,NaN,714.0,7.0,NaN,NaN,NaN,...,zCkv,P6pu4Vl,LM8l689qOp,NaN,ELof,xb3V,ZI9m,R4y5gQQWY8OodqDV,am7c,NaN
9996,NaN,NaN,NaN,NaN,NaN,812.0,7.0,NaN,NaN,NaN,...,oslk,sXbT3Cb,LM8l689qOp,NaN,ELof,WqMG,RAYp,55YFVY9,mj86,NaN
9997,NaN,NaN,NaN,NaN,NaN,819.0,7.0,NaN,NaN,NaN,...,oslk,05jAV0N,M_8D,NaN,ELof,TNEC,RAYp,55YFVY9,am7c,NaN


In [ ]:
df_test.drop(cols_nan, axis=1, inplace=True)

In [ ]:
df_test = si.transform(df_test)
df_test = pd.DataFrame(df_test, columns=cols)

In [ ]:
df_test

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,0.0,0.0,0.0,0.0,0.0,1225.0,7.0,0.0,0.0,8.0,...,UF16siJ,zCkv,APgdzOv,jySVZNlOJy,4n2X,ELof,xb3V,6fzt,Zy3gnGM,am7c
1,0.0,0.0,0.0,0.0,0.0,896.0,14.0,0.0,0.0,8.0,...,ot6oLzk,oslk,IIvC99a,LM8l689qOp,4n2X,ELof,xb3V,RAYp,F2FyR07IdsN7I,am7c
2,0.0,0.0,0.0,0.0,0.0,791.0,7.0,0.0,0.0,8.0,...,6VLNqhB,oslk,6YSocsg,LM8l689qOp,4n2X,kG3k,rgKb,RAYp,F2FyR07IdsN7I,mj86
3,0.0,0.0,0.0,0.0,0.0,2296.0,7.0,0.0,0.0,8.0,...,tzp8jNM,oslk,5nQ7A2G,jySVZNlOJy,4n2X,kG3k,rgKb,RAYp,F2FyR07IdsN7I,am7c
4,8.0,0.0,0.0,0.0,0.0,0.0,7.0,28.0,0.0,8.0,...,EPImS85,oslk,MI8s5nE,LM8l689qOp,4n2X,ELof,7P5s,RAYp,F2FyR07IdsN7I,am7c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,0.0,0.0,0.0,714.0,7.0,0.0,0.0,8.0,...,ch2oGfM,zCkv,P6pu4Vl,LM8l689qOp,4n2X,ELof,xb3V,ZI9m,R4y5gQQWY8OodqDV,am7c
9996,0.0,0.0,0.0,0.0,0.0,812.0,7.0,0.0,0.0,8.0,...,8d5YWu3,oslk,sXbT3Cb,LM8l689qOp,4n2X,ELof,WqMG,RAYp,55YFVY9,mj86
9997,0.0,0.0,0.0,0.0,0.0,819.0,7.0,0.0,0.0,8.0,...,WdPoLzk,oslk,05jAV0N,M_8D,4n2X,ELof,TNEC,RAYp,55YFVY9,am7c
9998,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,8.0,...,EMvv4zN,oslk,RDY7kpB,LM8l689qOp,4n2X,ELof,uWr3,RAYp,55YFVY9,am7c


In [ ]:
df_train_part

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,0.0,0.0,0.0,0.0,171.0,0.0,1.0,0.0,134.0,0.0,...,2606.0,4.0,1781.0,0.0,0.0,0.0,19.0,2.0,7.0,0.0
1,0.0,0.0,0.0,0.0,0.0,73.0,0.0,0.0,0.0,0.0,...,3251.0,3.0,2892.0,0.0,0.0,0.0,13.0,4.0,7.0,0.0
2,0.0,0.0,0.0,0.0,0.0,122.0,1.0,0.0,0.0,0.0,...,3342.0,4.0,1023.0,0.0,0.0,0.0,9.0,2.0,7.0,0.0
3,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,...,3561.0,4.0,1354.0,3.0,0.0,1.0,12.0,2.0,2.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,0.0,0.0,...,2949.0,4.0,3407.0,0.0,0.0,0.0,7.0,2.0,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34995,0.0,0.0,0.0,0.0,0.0,148.0,1.0,0.0,0.0,0.0,...,310.0,6.0,2336.0,0.0,0.0,0.0,9.0,3.0,13.0,0.0
34996,0.0,0.0,0.0,0.0,0.0,219.0,1.0,0.0,0.0,0.0,...,2003.0,4.0,227.0,0.0,0.0,0.0,7.0,2.0,7.0,0.0
34997,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,...,2379.0,4.0,1196.0,0.0,0.0,0.0,10.0,2.0,7.0,0.0
34998,0.0,0.0,0.0,0.0,0.0,111.0,1.0,0.0,0.0,0.0,...,310.0,2.0,2336.0,3.0,0.0,0.0,10.0,3.0,9.0,1.0


In [ ]:
tmp = oe.transform(df_test)
tmp1 = pd.DataFrame(tmp, columns=cols)

In [ ]:
tmp1

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,0.0,0.0,0.0,0.0,0.0,1225.0,-1.0,0.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,0.0,0.0,0.0,0.0,0.0,896.0,-1.0,0.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,0.0,0.0,0.0,0.0,0.0,791.0,-1.0,0.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,8.0,0.0,0.0,0.0,0.0,0.0,-1.0,28.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,0.0,0.0,0.0,714.0,-1.0,0.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9996,0.0,0.0,0.0,0.0,0.0,812.0,-1.0,0.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9997,0.0,0.0,0.0,0.0,0.0,819.0,-1.0,0.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9998,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
